In [ ]:
#Code taking too much time to run

In [ ]:
!unzip flowers.zip -d flowers

Archive:  flowers.zip
error [flowers.zip]:  missing 15728640 bytes in zipfile
  (attempting to process anyway)
error: invalid zip file with overlapped components (possible zip bomb)


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, roc_curve, auc, accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 1: ImageDataGenerator for loading and preprocessing images
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


val_generator = datagen.flow_from_directory(
    '/content/flowers/flowers',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

train_generator = datagen.flow_from_directory(
    '/content/flowers/flowers',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 860 images belonging to 5 classes.
Found 3457 images belonging to 5 classes.


In [ ]:
# Step 2: Initialize weights and biases
input_shape = (128, 128, 3)
hidden_size1 = 128
hidden_size2 = 64
output_size = train_generator.num_classes

# Define weights and biases
biases = {
    'b1': tf.Variable(tf.random.normal([hidden_size1])),
    'b2': tf.Variable(tf.random.normal([hidden_size2])),
    'b3': tf.Variable(tf.random.normal([output_size]))
}

weights = {
    'W1': tf.Variable(tf.random.normal([input_shape[0] * input_shape[1] * input_shape[2], hidden_size1])),
    'W2': tf.Variable(tf.random.normal([hidden_size1, hidden_size2])),
    'W3': tf.Variable(tf.random.normal([hidden_size2, output_size]))
}

In [ ]:
# Step 3 ,  4: Implement forward and backward passes
def forward_pass(X, weights, biases):
    X_flat = tf.reshape(X, [-1, input_shape[0] * input_shape[1] * input_shape[2]])

    layer1 = tf.add(tf.matmul(X_flat, weights['W1']), biases['b1'])
    layer1 = tf.nn.relu(layer1)

    layer2 = tf.add(tf.matmul(layer1, weights['W2']), biases['b2'])
    layer2 = tf.nn.relu(layer2)

    output_layer = tf.add(tf.matmul(layer2, weights['W3']), biases['b3'])
    output_layer = tf.nn.softmax(output_layer)

    return output_layer

def compute_loss(y_true, y_pred):
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), axis=1))

def backward_pass(X, y_true, weights, biases, learning_rate=0.01):
    with tf.GradientTape() as tape:
        y_pred = forward_pass(X, weights, biases)
        loss = compute_loss(y_true, y_pred)

    gradients = tape.gradient(loss, [weights['W1'], weights['W2'], weights['W3'],
                                     biases['b1'], biases['b2'], biases['b3']])

    # Update weights and biases
    weights['W1'].assign_sub(learning_rate * gradients[0])
    weights['W2'].assign_sub(learning_rate * gradients[1])
    weights['W3'].assign_sub(learning_rate * gradients[2])
    biases['b1'].assign_sub(learning_rate * gradients[3])
    biases['b2'].assign_sub(learning_rate * gradients[4])
    biases['b3'].assign_sub(learning_rate * gradients[5])

    return loss.numpy()

In [ ]:
# Step 5: Train the MLP
epochs = 10
train_losses = []
val_losses = []

for epoch in range(epochs):
    train_loss = 0
    for X_batch, y_batch in train_generator:
        loss = backward_pass(X_batch, y_batch, weights, biases)
        train_loss += loss
    train_losses.append(train_loss / len(train_generator))

    val_loss = 0
    for X_val, y_val in val_generator:
        loss = compute_loss(y_val, forward_pass(X_val, weights, biases)).numpy()
        val_loss += loss
    val_losses.append(val_loss / len(val_generator))

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]}, Val Loss: {val_losses[-1]}")

KeyboardInterrupt: 

In [ ]:
# Step 6: Evaluate the model
test_generator = datagen.flow_from_directory(
    '/content/flowers/flowers',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
y_pred_prob = forward_pass(test_generator[0][0], weights, biases).numpy()
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(test_generator[0][1], axis=1)
print(classification_report(y_true, y_pred))

In [ ]:
# Step 7: Plot ROC curve and calculate AUC
fpr = {}
tpr = {}
roc_auc = {}

for i in range(output_size):
    fpr[i], tpr[i], _ = roc_curve((y_true == i).astype(int), y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
for i in range(output_size):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (area = {roc_auc[i]:.2f})')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()